In [5]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import load_model
import pickle

In [6]:
# loading the trained model
model = load_model('model.h5')

In [7]:
# load the encoders and scaler

In [8]:
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_oncoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file: 
    scaler = pickle.load(file)

In [9]:
# example input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [10]:
# Framing the input data
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
# Label encoder on Gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
# One Hot Encoder On Geography
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns= onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\Neerao\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
# concat with one hot encoding
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
# scaling input
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.52281016,  0.91241915,  0.10281024, -0.69598177, -0.26422114,
         0.80773656,  0.64609167,  0.97024255, -0.87101922,  0.99720391,
        -0.57873591, -0.57380915]])

In [15]:
# Prediction 
prediction  = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


array([[0.07238936]], dtype=float32)

In [16]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.072389364)

In [21]:
if prediction_proba > 0.5:
    print(f'Probability to Churn: {prediction_proba*100:.2f}')
    print('The customer is likely to churn.')
else:
    print(f'Probability to Churn: {prediction_proba*100:.2f}')
    print('The customer is not likely to churn.')
    

Probability to Churn: 7.24
The customer is not likely to churn.


In [ ]:
# Thank You...